### Load Data

In [13]:
import pandas as pd

In [14]:
sales = pd.read_csv('ventas.csv')
sales = sales.astype(str)
sales.head()

,CustomerId,material,calmonth,uni_box
0,499920078,9151,201909,0.4364
1,499920078,2287,201909,3.1701
2,499920078,4526,201909,0.2818
3,499920078,14050,201909,0.2642
4,499920078,1333,201909,2.1134


In [15]:
customers = pd.read_csv('customers_sampled.csv')
customers.head()

,CustomerId,pc_agr_300m,pc_comercial_300m,pc_generales_300m,pc_habitacional_300m,pc_habitacional_mixta_300m,pc_industrial_300m,pc_minero_300m,pc_mixta_300m,pc_negocios_300m,...,velocidad_hora_19,velocidad_hora_20,velocidad_hora_21,velocidad_hora_22,velocidad_hora_23,accesibilidad,socioeconomic_status_rgm,industry_customer_size,rgm_channel,sub_canal_comercial
0,499920078,0.0,0.00,6.11,48.40,37.71,6.28,0.0,0.0,0.0,...,24.0,24.0,26.0,17.0,11.0,0.184550,HOGAR,3,TRADICIONAL,Estanquillos / kioscos
1,499920499,0.0,0.00,0.00,89.38,6.39,4.23,0.0,0.0,0.0,...,22.0,34.0,60.0,10.0,6.0,0.265247,TRANSITO,4,TRADICIONAL,Abarrotes / Almacenes / Bodegas / Víveres
2,499921473,0.0,1.17,15.51,66.28,16.65,0.39,0.0,0.0,0.0,...,40.0,40.0,28.0,20.0,29.0,0.191479,HOGAR,5,TRADICIONAL,Estanquillos / kioscos
3,499921557,0.0,0.00,81.14,16.57,1.99,0.00,0.0,0.0,0.0,...,30.0,40.0,13.0,40.0,32.0,0.251428,TRANSITO,5,TRADICIONAL,Abarrotes / Almacenes / Bodegas / Víveres
4,499921908,0.0,0.00,0.00,100.00,0.00,0.00,0.0,0.0,0.0,...,13.0,30.0,30.0,30.0,30.0,0.276372,HOGAR,4,TRADICIONAL,Estanquillos / kioscos


In [16]:
products = pd.read_csv('20230223_productos.csv')
products.head()

,Material,Material_desc,Productos_Por_Empaque,BrandPresRet,ProdKey,Brand,Presentation,MLSize,Returnability,Pack,...,Container,Ncb,ProductType,ProductCategory,SegAg,SegDet,GlobalCategory,GlobalSubcategory,BrandGrouper,GlobalFlavor
0,9465,CIEL EXPRIM LIMON 600 ML NR 6B,6,CIEL EXPRIM 600 ML NO RETORNABLE,AGUA SABORIZADA,CIEL EXPRIM,600 ML NR,600,NO RETORNABLE,600 ML,...,PLASTICO,1,AGUA SABORIZADA,AGUA,AGUA SABORIZADA,AGUA SABORIZADA,AGUA,AGUA SABORIZADA,CIEL,LIMÓN
1,14450,PWD FIT ARNDNO ACAI 500ML NR PET 6B,6,POWERADE FIT 500 ML NO RETORNABLE,ISOTONICOS,POWERADE FIT,500 ML NR PET,500,NO RETORNABLE,500 ML,...,PLASTICO,1,ISOTONICOS,CATEGORIAS EN EXPANSION,ISOTONICOS,ISOTONICOS LIGHT,BEBIDAS EMERGENTES,ISOTÓNICOS,POWERADE,ARÁNDANO ACAÍ
2,148,FANTA UVA 500 ML RET VID 24B,24,FANTA 500 ML RETORNABLE,SABORES INDIVIDUAL RETORNABLE,FANTA,500 ML RET,500,RETORNABLE,500 ML,...,VIDRIO,0,SABORES REGULAR,REFRESCOS,SABORES,SABORES REGULAR,REFRESCOS,SABORES,FANTA,UVA
3,1088,SIERRAZUL AP 3.8 LTS NR 4B,4,SIERRAZUL AGUA PURIFICADA 3.80 L NO RETORNABLE,AGUA PURIFICADA FAMILIAR,SIERRAZUL AGUA PURIFICADA,GALÓN 3.8 LTS.,3800,NO RETORNABLE,GALÓN,...,PLASTICO,1,AGUA PURIFICADA,AGUA,AGUA PURIFICADA,AGUA PURIFICADA,AGUA,AGUA PURIFICADA,SIERRAZUL,SIN SABOR
4,2142,TOPO CHICO AM 340 ML (12OZ) NR 6B,6,TOPO CHICO A.M. 340 ML NO RETORNABLE,AGUA MINERAL INDIVIDUAL,TOPO CHICO A.M.,12 OZ. NR VIDRIO,340,NO RETORNABLE,12 OZ.,...,VIDRIO,0,AGUA MINERAL,AGUA,AGUA MINERAL,AGUA MINERAL,AGUA,AGUA MINERAL,TOPO CHICO,SIN SABOR


### Feature Engineering | Success

In [17]:
def longest_consecutive_months(months):
    months = sorted(set(months))
    max_consecutive = 1
    current_streak = 1
    


    for i in range(1, len(months)):
        prev_year, prev_month = int(months[i-1][:4]), int(months[i-1][4:])
        curr_year, curr_month = int(months[i][:4]), int(months[i][4:])
        
        # Check if the current month is consecutive
        if (curr_year == prev_year and curr_month == prev_month + 1) or (curr_year == prev_year + 1 and curr_month == 1 and prev_month == 12):
            current_streak += 1
            max_consecutive = max(max_consecutive, current_streak)
        else:
            current_streak = 1

    return max_consecutive

# Group by CustomerId and material, and calculate the earliest, latest, and longest consecutive calmonths
success_dataframe = sales.groupby(['CustomerId', 'material']).agg(
    earliest_calmonth=('calmonth', 'min'),
    latest_calmonth=('calmonth', 'max'),
    longest_consecutive_months=('calmonth', longest_consecutive_months)
).reset_index()

success_dataframe.drop(['earliest_calmonth','latest_calmonth'], axis = 1, inplace = True)
success_dataframe['Success'] = success_dataframe['longest_consecutive_months'] >= 5

In [18]:
success_dataframe.head()

,CustomerId,material,longest_consecutive_months,Success
0,499920078,1,1,False
1,499920078,100,40,True
2,499920078,101,1,False
3,499920078,1012,11,True
4,499920078,117,1,False


### Feature Selection | Customers | Numerical Variables

In [25]:
customers_numeric = customers.drop(['socioeconomic_status_rgm', 'rgm_channel', 'sub_canal_comercial'], axis = 1)
customers_numeric['CustomerId'] = customers_numeric['CustomerId'].astype(str)

In [26]:
merged_customer_data = pd.merge(success_dataframe, customers_numeric, on='CustomerId', how='left')
merged_customer_data.head()

,CustomerId,material,longest_consecutive_months,Success,pc_agr_300m,pc_comercial_300m,pc_generales_300m,pc_habitacional_300m,pc_habitacional_mixta_300m,pc_industrial_300m,...,velocidad_hora_16,velocidad_hora_17,velocidad_hora_18,velocidad_hora_19,velocidad_hora_20,velocidad_hora_21,velocidad_hora_22,velocidad_hora_23,accesibilidad,industry_customer_size
0,499920078,1,1,False,0.0,0.0,6.11,48.4,37.71,6.28,...,23.0,18.0,21.0,24.0,24.0,26.0,17.0,11.0,0.18455,3
1,499920078,100,40,True,0.0,0.0,6.11,48.4,37.71,6.28,...,23.0,18.0,21.0,24.0,24.0,26.0,17.0,11.0,0.18455,3
2,499920078,101,1,False,0.0,0.0,6.11,48.4,37.71,6.28,...,23.0,18.0,21.0,24.0,24.0,26.0,17.0,11.0,0.18455,3
3,499920078,1012,11,True,0.0,0.0,6.11,48.4,37.71,6.28,...,23.0,18.0,21.0,24.0,24.0,26.0,17.0,11.0,0.18455,3
4,499920078,117,1,False,0.0,0.0,6.11,48.4,37.71,6.28,...,23.0,18.0,21.0,24.0,24.0,26.0,17.0,11.0,0.18455,3


In [27]:
from mrmr import mrmr_classif
selected_features = mrmr_classif(X=merged_customer_data.drop(['CustomerId', 'material', 'longest_consecutive_months', 'Success'], axis = 1), y=merged_customer_data['Success'], K=10)

100%|██████████| 10/10 [00:16<00:00,  1.69s/it]


In [28]:
selected_features

['industry_customer_size',
 'pob_e_300m',
 'pc_habitacional_mixta_300m',
 'ingreso_maximo_300m',
 'pob_ab_300m',
 'prob_inter_mod_300m',
 'flo_finde_cmas_300m',
 'flo_sem_cmas_300m',
 'autos_hora_18',
 'ingreso_rentas_300m']

In [30]:
# Calculate the correlation matrix
corr_matrix = customers_numeric.corr()

# Convert the correlation matrix to a long format
corr_long = corr_matrix.unstack().reset_index()
corr_long.columns = ['Variable1', 'Variable2', 'Correlation']

# Filter out self-correlations
corr_long = corr_long[corr_long['Variable1'] != corr_long['Variable2']]

# Remove duplicate pairs by sorting and dropping duplicates
corr_long['Pairs'] = corr_long.apply(lambda row: tuple(sorted([row['Variable1'], row['Variable2']])), axis=1)
corr_long = corr_long.drop_duplicates(subset='Pairs').drop(columns='Pairs')

# Filter for correlations >= 0.7
strong_correlations = corr_long[abs(corr_long['Correlation']) >= 0.7]

# Sort by absolute correlation values
strong_correlations = pd.DataFrame(strong_correlations.sort_values(by='Correlation', ascending=False))

#strong_correlations.to_csv('Customer_Correlation_Numeric.csv')

In [34]:
# Check if entries in selected_features appear in combination with another entry in the list
combinations_found = []

for feature in selected_features:
    combinations = strong_correlations[
        (strong_correlations['Variable1'] == feature) & (strong_correlations['Variable2'].isin(selected_features)) |
        (strong_correlations['Variable2'] == feature) & (strong_correlations['Variable1'].isin(selected_features))
    ]
    if not combinations.empty:
        combinations_found.append((feature, combinations))

# Print results
for feature, combinations in combinations_found:
    #print(f"{feature} appears in combination with another entry in selected_features:")
    #print(combinations)
    #print('\n')
    if feature in selected_features: selected_features.remove(feature)

selected_features

['industry_customer_size',
 'pob_e_300m',
 'pc_habitacional_mixta_300m',
 'ingreso_maximo_300m',
 'pob_ab_300m',
 'prob_inter_mod_300m',
 'autos_hora_18',
 'ingreso_rentas_300m']

### Feature Selection | Customers | Categorical

In [35]:
customers_categoric = customers[['socioeconomic_status_rgm', 'rgm_channel', 'sub_canal_comercial']]
customers_categoric.head()

,socioeconomic_status_rgm,rgm_channel,sub_canal_comercial
0,HOGAR,TRADICIONAL,Estanquillos / kioscos
1,TRANSITO,TRADICIONAL,Abarrotes / Almacenes / Bodegas / Víveres
2,HOGAR,TRADICIONAL,Estanquillos / kioscos
3,TRANSITO,TRADICIONAL,Abarrotes / Almacenes / Bodegas / Víveres
4,HOGAR,TRADICIONAL,Estanquillos / kioscos


In [38]:
import os as os
from itertools import product
import numpy as np
import scipy.stats as ss

In [40]:
customers_categoric.dropna()
customers_categoric1 = customers_categoric.columns
customers_categoric2 = customers_categoric.columns
customers_categoric_prod = list(product(customers_categoric1,customers_categoric2, repeat = 1))
customers_categoric_prod

[('socioeconomic_status_rgm', 'socioeconomic_status_rgm'),
 ('socioeconomic_status_rgm', 'rgm_channel'),
 ('socioeconomic_status_rgm', 'sub_canal_comercial'),
 ('rgm_channel', 'socioeconomic_status_rgm'),
 ('rgm_channel', 'rgm_channel'),
 ('rgm_channel', 'sub_canal_comercial'),
 ('sub_canal_comercial', 'socioeconomic_status_rgm'),
 ('sub_canal_comercial', 'rgm_channel'),
 ('sub_canal_comercial', 'sub_canal_comercial')]

In [41]:
result = []
for i in customers_categoric_prod:
    if i[0] != i[1]:
        result.append((i[0],i[1],list(ss.chi2_contingency(pd.crosstab(customers_categoric[i[0]], customers_categoric[i[1]])))[1]))
result

[('socioeconomic_status_rgm', 'rgm_channel', 1.0),
 ('socioeconomic_status_rgm', 'sub_canal_comercial', 1.3983003688076884e-17),
 ('rgm_channel', 'socioeconomic_status_rgm', 1.0),
 ('rgm_channel', 'sub_canal_comercial', 1.0),
 ('sub_canal_comercial', 'socioeconomic_status_rgm', 1.398300368807669e-17),
 ('sub_canal_comercial', 'rgm_channel', 1.0)]

In [42]:
chi_test_output = pd.DataFrame(result, columns = ['var1', 'var2', 'coeff'])
## Using pivot function to convert the above DataFrame into a crosstab
chi_test_output.pivot(index='var1', columns='var2', values='coeff')


var2,rgm_channel,socioeconomic_status_rgm,sub_canal_comercial
var1,,,
rgm_channel,NaN,1.000000e+00,1.000000e+00
socioeconomic_status_rgm,1.0,NaN,1.398300e-17
sub_canal_comercial,1.0,1.398300e-17,NaN
